In [2]:
import nltk
from nltk.tokenize import word_tokenize 
import pandas as pd
import os
import json
import glob
import spacy
nlp = spacy.load('pt')



jsons=[]
for filename in glob.iglob("D:\\socialbus\\users\\*" + '*', recursive=True):
    try:
        jsons.append(pd.read_json(filename,lines=True))
       # print(filename)
        #print("deu")
    except:
        pass
        
print(str(len(jsons)) + " json files found")  

46 json files found


In [16]:
i=0
for json in jsons:
    i=len(json)-1
    while(i>1):
        
        
        
        print(json['text'][i])
        print("Time of Comment: " + str(get_time_of_comment(json['created_at'][i])))
        print("Number of Stopwords: "+ str(get_stopword_nr(json['text'][i])))
        print("Number of Hashtags: "+ str(get_nr_of_hashtags(json['metadata'][i])))
        print("Number of Mentions: "+ str(get_nr_of_mentions(json['metadata'][i])))
        print("Number of Urls: "+ str(get_nr_of_urls(json['metadata'][i])))
        print("Number of Emoticons: "+ str(get_nr_of_emoticons(json['metadata'][i])))
        print("Number of Swears: "+ str(get_swear_count(json['text'][i])))
        print("Number of Favorites: "+ str(get_favorites_count(json['favorite_count'][i])))
        print("Number of Verbs: "+ str(get_verb_count(json['text'][i])))
        print("Number of Nouns: "+ str(get_noun_count(json['text'][i])))
        print("Number of Adjectives: "+ str(get_adj_count(json['text'][i])))
        print("Tweet Length "+ str(get_tweet_length(json['text'][i])))
              
        print("\n -------------------------------------------------------\n")
        
        
        
        
        i=i-1
   




#get_nr_of_hashtags(jsonLine)        
#get_nr_of_mentions(jsonLine) 
#get_nr_of_urls(jsonLine) 
#get_nr_of_emoticons(jsonLine) 
#get_swear_count(jsonLine) 
#get_favorites_count(jsonLine) 
#get_verb_count(jsonLine) 
#get_noun_count(jsonLine) 
#get_adj_count(jsonLine) 

RT @rubenbranco123: Porra, os Nenucos destes tempos novos são diferentes.
Time of Comment: 17:15:34
Number of Stopwords: 2
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 0
Number of Adjectives: 2
Tweet Length 73

 -------------------------------------------------------

RT @melanievicente6: Desejosa que chegue o verão para poder deixar secar o meu cabelo ao natural
Time of Comment: 17:15:32
Number of Stopwords: 6
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 3
Number of Nouns: 2
Number of Adjectives: 0
Tweet Length 96

 -------------------------------------------------------

@dassmariaduarte foi +/- o que lhe disse, saiu-me "é-me difícil respeitar alguém que rotula outra pessoa só por cau… https://t.co/N88j1xjnCJ
Time of Comment: 17:15:31
Number of Stopwords: 7
Number of H

Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 0
Number of Adjectives: 1
Tweet Length 140

 -------------------------------------------------------

RT @aCartilhaDoColo: É um crime o que estão a fazer ao Sporting, e por inerência, ao futebol nacional. Isto que fizeram hoje ao Silas, é ma…
Time of Comment: 17:14:53
Number of Stopwords: 12
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 5
Number of Adjectives: 2
Tweet Length 140

 -------------------------------------------------------

RT @KikoisHot: o meu amigo meteu esta música da gaga e tive de ir a correr buscar uma peruca loira barata. https://t.co/9cOvtej3iM
Time of Comment: 17:14:52
Number of Stopwords: 9
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Em


RT @JoaoFelixDaCos1: as pessoas não tem noção do estado do nosso país...tds os negócios novos que analisei nos últimos 5 anos, são para n f…
Time of Comment: 17:14:22
Number of Stopwords: 11
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 5
Number of Nouns: 3
Number of Adjectives: 2
Tweet Length 140

 -------------------------------------------------------

@BrunodC72 Sem apelo nem agravo, até que a voz te doa!
Time of Comment: 17:14:22
Number of Stopwords: 5
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 4
Number of Adjectives: 0
Tweet Length 54

 -------------------------------------------------------

RT @eujafizaquiloou: Filipinos brancos ou negros

Negros-fav

Brancos-Rt
Time of Comment: 17:14:20
Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 1
Number 

Number of Verbs: 2
Number of Nouns: 2
Number of Adjectives: 0
Tweet Length 47

 -------------------------------------------------------

LISBOA | Próximo de: Radar marcado para hoje pela PSP. Rua Dr. Armando dos Santos Ferreira – Santo António Cavaleir… https://t.co/e9nmNEbQBc
Time of Comment: 17:14:03
Number of Stopwords: 4
Number of Hashtags: 0
Number of Mentions: 0
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 0
Number of Adjectives: 0
Tweet Length 140

 -------------------------------------------------------

@capasderevistas Por favor, quais eram os problemas hahahhaha
Time of Comment: 17:14:03
Number of Stopwords: 2
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 3
Number of Adjectives: 0
Tweet Length 61

 -------------------------------------------------------

RT @AgirOfficial: Até a nossa m

Number of Stopwords: 2
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 2
Number of Adjectives: 2
Tweet Length 140

 -------------------------------------------------------

RT @Yoko1904: "patrocinar" um clube é muito mais que ter o nome na camisola. Perceber a identidade, as bases de fundação e a sua presença s…
Time of Comment: 17:13:49
Number of Stopwords: 13
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 6
Number of Adjectives: 0
Tweet Length 140

 -------------------------------------------------------

RT @AgirOfficial: Até a nossa morte vai ser MADE IN CHINA.
Time of Comment: 17:13:48
Number of Stopwords: 2
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1

...

Number of Verbs: 1
Number of Nouns: 0
Number of Adjectives: 0
Tweet Length 30

 -------------------------------------------------------

RT @observadorpt: Do apocalipse Balenciaga ao desfile que reabriu o Louvre. A Semana da Moda de Paris em 16 atos: https://t.co/MLM2FebFqu
Time of Comment: 17:09:19
Number of Stopwords: 6
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 1
Number of Nouns: 2
Number of Adjectives: 0
Tweet Length 137

 -------------------------------------------------------

O mistério de Florencia Kirchner https://t.co/mD8R2AGtpo
Time of Comment: 17:09:16
Number of Stopwords: 1
Number of Hashtags: 0
Number of Mentions: 0
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 0
Number of Nouns: 1
Number of Adjectives: 0
Tweet Length 56

 -------------------------------------------------------

A @hollyacollie ficou de rastos após um

Number of Adjectives: 1
Tweet Length 58

 -------------------------------------------------------

@Luis_Eusebio @1963Cor Mas para ti só as iniciativas legislativas do PS é que contam na AR? Oh pá, há vários partid… https://t.co/mR89ra4Lru
Time of Comment: 17:59:54
Number of Stopwords: 8
Number of Hashtags: 0
Number of Mentions: 2
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 3
Number of Adjectives: 2
Tweet Length 140

 -------------------------------------------------------

o problema de fazer sexo com luz apagada, é que nunca sabes se ela está excitada ou com o período https://t.co/uAflSLCYSm
Time of Comment: 17:59:51
Number of Stopwords: 11
Number of Hashtags: 0
Number of Mentions: 0
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 6
Number of Nouns: 5
Number of Adjectives: 0
Tweet Length 121

 -------------------------------------------------------

@SICNot

Number of Nouns: 2
Number of Adjectives: 1
Tweet Length 55

 -------------------------------------------------------

@yourcenar1234 Da maneira que te queixas até estou admirado dele não te chamar ASAE!
Time of Comment: 17:59:19
Number of Stopwords: 7
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 3
Number of Adjectives: 0
Tweet Length 84

 -------------------------------------------------------

RT @captomente: O plano de Varandas &amp; C.ia até ao momento:

- vender Raphinha
- vender Bas Dost
- vender Bruno Fernandes
- vender/emprestar…
Time of Comment: 17:59:16
Number of Stopwords: 3
Number of Hashtags: 0
Number of Mentions: 1
Number of Urls: 0
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 4
Number of Nouns: 2
Number of Adjectives: 0
Tweet Length 144

 -------------------------------------------------------

RT @AAAAAGGHHHH: https:

Number of Nouns: 4
Number of Adjectives: 1
Tweet Length 107

 -------------------------------------------------------

@Ricardo37617644 @VoltPortugal @AndreCAVentura Já foram entregues há 4 meses 😉

Estamos à espera de confirmação ofi… https://t.co/bpagXbwgWo
Time of Comment: 17:58:50
Number of Stopwords: 4
Number of Hashtags: 0
Number of Mentions: 3
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 3
Number of Adjectives: 0
Tweet Length 140

 -------------------------------------------------------

@PNSpedronuno @JoedwartJoe E porque será que os empresários estão sempre ricos e a empresa e os trabalhadores não g… https://t.co/FnAkfhZPuo
Time of Comment: 17:58:49
Number of Stopwords: 9
Number of Hashtags: 0
Number of Mentions: 2
Number of Urls: 1
Number of Emoticons: 0
Number of Swears: 0
Number of Favorites: 0
Number of Verbs: 2
Number of Nouns: 1
Number of Adjectives: 1
Tweet Length 140

 --------------------------

KeyboardInterrupt: 

In [3]:
def get_stopword_nr(sentence):
    
    word_tokens = word_tokenize(sentence)
    count=0
    
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for stopword in stopwords:
        for word in word_tokens:
            if stopword==word:
                count+=1
       
    
    return count
    
    

In [4]:
def get_time_of_comment(jsonLine):
     
    dateStr=str(jsonLine)
       
    fullDateArray=dateStr.split()
    time=fullDateArray[1].split('+')
    
    return time[0]
    

In [5]:
def get_nr_of_hashtags(jsonLine):
    return len(jsonLine['hashtags'])

In [6]:
def get_nr_of_mentions(jsonLine):
    return len(jsonLine['mentions'])

In [7]:
def get_nr_of_urls(jsonLine):
    return len(jsonLine['urls'])

In [8]:
def get_nr_of_emoticons(jsonLine):
    return len(jsonLine['emoticons'])

In [9]:
def get_swear_count(jsonLine):
    swearCount=0
    swears=['broch','cabrões','cabr','cagalhão','caralh','cona','corn','enrabar','cuzinho','esporra','fod','mamadas','panasca','peid','piça','picha','pila','tomates','puta','merd','caralho','crl','fds','fdç','paneleir']
    for swear in swears:
        if swear.lower() in jsonLine.lower():
            swearCount+=1
            
    return swearCount        

In [10]:
def get_favorites_count(jsonLine):
    return jsonLine

In [11]:
def get_day_of_tweet(jsonLine):
    dateStr=str(jsonLine)
       
    fullDateArray=dateStr.split()

In [12]:
def get_verb_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    verbCount=0


    for token in doc:
        if token.pos_=='VERB':
            verbCount+=1
        
    return verbCount

In [13]:
def get_noun_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    nounCount=0


    for token in doc:
        if token.pos_=='NOUN' or token.pos=='PROPN':
            nounCount+=1
        
    return nounCount

In [14]:
def get_adj_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    adjCount=0


    for token in doc:
        if token.pos_=='ADJ':
            adjCount+=1
        
    return adjCount

In [15]:
def get_tweet_length(jsonLine):
    return len(jsonLine)